# SNN that detects High Frequency Oscillations (HFOs) with constant parameters
This notebook is a simple example of how to use a Spiking Neural Network (SNN) to detect HFOs

### What is an HFO?
High Frequency Oscillations (HFOs) are a type of brain activity that occurs in the range of 80-500 Hz. They are believed to be related to the generation of seizures in patients with epilepsy. The detection of HFOs is an important task in the diagnosis and treatment of epilepsy. 

In terms of electrophysiology, HFOs are characterized by their high frequency and short duration, often lasting only a few milliseconds. The wave of a typical HFO consists of at least 4 UP and DOWN waves.

In [271]:
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
import numpy as np

LIF?

Init signature: LIF(*args, **kwargs)
Docstring:     
Leaky-Integrate-and-Fire (LIF) neural Process.

LIF dynamics abstracts to:
u[t] = u[t-1] * (1-du) + a_in         # neuron current
v[t] = v[t-1] * (1-dv) + u[t] + bias  # neuron voltage
s_out = v[t] > vth                    # spike if threshold is exceeded
v[t] = 0                              # reset at spike

Parameters
----------
shape : tuple(int)
    Number and topology of LIF neurons.
u : float, list, numpy.ndarray, optional
    Initial value of the neurons' current.
v : float, list, numpy.ndarray, optional
    Initial value of the neurons' voltage (membrane potential).
du : float, optional
    Inverse of decay time-constant for current decay. Currently, only a
    single decay can be set for the entire population of neurons.
dv : float, optional
    Inverse of decay time-constant for voltage decay. Currently, only a
    single decay can be set for the entire population of neurons.
bias_mant : float, list, numpy.ndarray, optiona

### Check WD (change if necessary) and file loading

In [272]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn


## Create the Custom Input Layer

### Define function to read the input data from the csv file and generate the corresponding spike events

In [273]:
import pandas as pd

def read_spike_events(file_path: str):
    """Reads the spike events from the input file and returns them as a numpy array

    Args:
        file_path (str): name of the file containing the spike events
    """
    spike_events = []

    try:
        # Read the spike events from the file
        df = pd.read_csv(file_path, header=None)

        # Detect errors
        if df.empty:
            raise Exception("The input file is empty")

        # Convert the scientific notation values to integers if any exist
        df = df.applymap(lambda x: int(float(x)) if (isinstance(x, str) and 'e' in x) else x)

        # Convert the dataframe to a numpy array
        spike_events = df.to_numpy()
        return spike_events[0]
    except Exception as e:
        print("Unable to read the input file: ", file_path, " error:", e)

    return spike_events

## Configurable Parameters

In [274]:
from utils.input import BaselineAlgorithm, MarkerType, ModelDistStrategy

# Declare if using ripples, fast ripples, or both
chosen_band = MarkerType.RIPPLE     # RIPPLE, FAST_RIPPLE, or BOTH

# Specify the chosen Baseline Algorithm
chosen_baseline_alg_suffix = BaselineAlgorithm.MEDIAN

# Select the Distribution Model Strategy
selected_strategy = ModelDistStrategy.MEAN_AND_STD

# Define the Weight Scale of the Dense Layer #TODO: Test changing this
weights_scale_input = 0.3   # 0.5

# Define the IQR ranges for the specific data being fed (synaptic time constants)
ripple_IQR = [0.48, 6.5]
fr_IQR = [0.49, 2.93] # Inter-Quartile Range for the time constants of the Fast-Ripple neurons

# Define the mean and std. deviation of the synaptic time constants
synaptic_tc_mean = 4.0335
synaptic_tc_std_dev = 8.1027

# Define the mean and std. deviation of the Membrane Potential Time Constants
mean_dv = 10    # Mean voltage time constant = 15ms (following Indiveri's paper)
std_dev_dv = 5  # Standard deviation of the voltage time constant. If 0, then the time constant is fixed 

# Range of random values to subtract from the excitatory time constants to get the inhibitory time constants
inh_subtract_range = [0.05, 1.0]  

### Load the UP and DOWN spikes from the CSV Files

In [275]:
# Define the name of the dataset version being used
DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"

#### Load the Baseline Thresholds from the output file from the baseline process

In [276]:
# Load the Baseline Thresholds
BASELINE_FILE = f"../signal_to_spike/baseline_results/{DATASET_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)

Ripple Thresholds:  2.4933 -2.4933
FR Thresholds:  1.3215 -1.3215


In [277]:
def band_to_thresholds(band: MarkerType):
    """Returns the ripple and FR thresholds for the given band

    Args:
        band (MarkerType): the band for which the thresholds are required
    """
    if band == MarkerType.RIPPLE:
        return ripple_thresh_up, ripple_thresh_down
    elif band == MarkerType.FAST_RIPPLE:
        return fr_thresh_up, fr_thresh_down
    else:
        raise Exception("Invalid band type")

In [278]:
from utils.input import read_spike_events, band_to_file_name
from utils.io import preview_np_array

# Define the path of the files containing the spike events
INPUT_PATH = f"../signal_to_spike/results/{DATASET_FILENAME}"
band_file_name = band_to_file_name(chosen_band)

# Define the chosen thresholds
thresh_up, thresh_down = band_to_thresholds(chosen_band)
print("Thresholds: ", thresh_up, thresh_down)

# Call the function to read the spike events
up_spikes_file_path = f"{INPUT_PATH}/{band_file_name}_up_spike_train_{thresh_up}.csv"
up_spike_train = read_spike_events(up_spikes_file_path)

down_spikes_file_path = f"{INPUT_PATH}/{band_file_name}_down_spike_train_{thresh_down}.csv"
down_spike_train = read_spike_events(down_spikes_file_path)

preview_np_array(up_spike_train, "up_spike_train")
preview_np_array(down_spike_train, "down_spike_train")

Thresholds:  2.4933 -2.4933
up_spike_train Shape: (7037, 2).
Preview: [[ 5.17578125e+01 -1.00000000e+00]
 [ 5.90820312e+01 -1.00000000e+00]
 [ 8.44726562e+01 -1.00000000e+00]
 [ 1.12792969e+02 -1.00000000e+00]
 [ 1.23046875e+02 -1.00000000e+00]
 ...
 [ 1.19835938e+05 -1.00000000e+00]
 [ 1.19845215e+05 -1.00000000e+00]
 [ 1.19855957e+05 -1.00000000e+00]
 [ 1.19951172e+05 -1.00000000e+00]
 [ 1.19975586e+05 -1.00000000e+00]]
down_spike_train Shape: (7069, 2).
Preview: [[ 5.46875000e+01 -1.00000000e+00]
 [ 5.66406250e+01 -1.00000000e+00]
 [ 8.15429688e+01 -1.00000000e+00]
 [ 1.06933594e+02 -1.00000000e+00]
 [ 1.20117188e+02 -1.00000000e+00]
 ...
 [ 1.19841309e+05 -1.00000000e+00]
 [ 1.19850586e+05 -1.00000000e+00]
 [ 1.19945312e+05 -1.00000000e+00]
 [ 1.19962891e+05 -1.00000000e+00]
 [ 1.19978027e+05 -1.00000000e+00]]


### Define the SpikeEvent Generator Interface

In [279]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import OutPort

class SpikeEventGen(AbstractProcess):
    """Input Process that generates spike events based on the input file

    Args:
        @out_shape (tuple): Shape of the output port
        @exc_spike_events (np.ndarray): Excitatory spike events
        @inh_spike_event (np.ndarray): Inhibitory spike events
        @name (str): Name of the process
    """
    def __init__(self, out_shape: tuple, exc_spike_events: np.ndarray, inh_spike_event: np.ndarray, name: str) -> None:
        super().__init__(name=name)
        self.s_out = OutPort(shape=out_shape)
        self.exc_spike_events = Var(shape=exc_spike_events.shape, init=exc_spike_events)
        self.inh_spike_events = Var(shape=inh_spike_event.shape, init=inh_spike_event)


## Define the Architecture of the Network

In [280]:
# Define the number of neurons in the Input Spike Event Generator
n_spike_gen = 2  # 2 neurons in the input spike event generator

# Define the number of neurons in each LIF Layer
n_lif1 = 256   # 256 neurons in the first LIF layer
# n2 = 1  # 1 neuron in the second layer

### Choose the LIF Models to use

In [281]:
use_refractory = False

### Define the LIF parameters

In [282]:
# Constants for the LIF Process
v_th = 1
v_init = 0

# LIF1 Process
dv1 = 0.07
du1 = 0.2  

### Create the LIF Processes

In [283]:
if not use_refractory:
    # Create LIF1 process
    lif1 = LIF(shape=(n_lif1,),  # There are 2 neurons
            vth=v_th,  # TODO: Verify these initial values
            v=v_init,
            dv=dv1,    # Inverse of decay time-constant for voltage decay
            du=du1,  # Inverse of decay time-constant for current decay
            bias_mant=0,
            bias_exp=0,
            name="lif1")

### Create the Refractory LIF Processes

In [284]:
from lava.proc.lif.process import LIFRefractory
from lava.magma.core.process.process import LogConfig
import logging

# Constants for the Refractory LIF Process
refrac_period = 20   # Number of time-steps for the refractory period

if use_refractory:
    # Create Refractory LIF1 process
    lif1 = LIFRefractory(shape=(n_lif1,),  # There are 2 neurons
            vth=v_th,  # TODO: Verify these initial values
            v=v_init,
            dv=dv1,    # Inverse of decay time-constant for voltage decay
            du=du1,  # Inverse of decay time-constant for current decay
            bias_mant=0,
            bias_exp=0,
            refractory_period=refrac_period,
            name="lif1",
            # log_config=LogConfig(level=logging.DEBUG, level_console=logging.DEBUG, logs_to_file=False)
            )

### Create a `ConfigTimeConstantsLIF` object

#### Define the time constants for the `ConfigTimeConstantsLIF` neurons
The synapse time constants, corresponding to `du_exc` and `du_inh` will be different for each neuron in the LIF layer. Likewise, the excitatory and inhibitory time constants will also differ for each neuron in order to capture the dynamics of the HFOs.

In [285]:
from utils.neuron_dynamics import time_constant_to_fraction
# Create the np arrays for the time constants of each neuron

# Select the chosen IQR based on the chosen band
chosen_IQR = ripple_IQR
if chosen_band == MarkerType.FAST_RIPPLE:
    chosen_IQR = fr_IQR
elif chosen_band != MarkerType.RIPPLE:
    raise Exception("What to do in this case?")


if selected_strategy == ModelDistStrategy.IQR:
    chosen_mu = np.mean(chosen_IQR)  # Midpoint of the IQR
    # Calculate the standard deviation of the normal distribution
    # For a normal distribution, the first quartile is ~0.675 standard deviations below the mean
    chosen_std_dev = (chosen_IQR[1] - chosen_IQR[0]) / (2 * 0.675)  # standard deviation is the IQR divided by 2*0.675
elif selected_strategy == ModelDistStrategy.MEAN_AND_STD:
    # Use the actual values from the STS_ANALYSIS
    chosen_mu = synaptic_tc_mean
    chosen_std_dev = synaptic_tc_std_dev

print("Chosen mu: ", chosen_mu)
print("Chosen std_dev: ", chosen_std_dev)
# print("Chosen IQR: ", chosen_IQR)

Chosen mu:  4.0335
Chosen std_dev:  8.1027


### Generate a random distribution of the Synaptic Excitatory time constants 

In [286]:
# Generate the time constants for the Fast-Ripple neurons
exc_syn_time_constants = np.random.normal(chosen_mu, chosen_std_dev, n_lif1)
# preview_np_array(exc_syn_time_constants, "exc_syn_time_constants", edge_items=10)
print("Min and max time constants before:", np.min(exc_syn_time_constants), np.max(exc_syn_time_constants))


# Cannot have negative time constants. Make them 0 or positive?
# exc_syn_time_constants = np.clip(exc_syn_time_constants, a_min=0, a_max=None)
exc_syn_time_constants = np.abs(exc_syn_time_constants)
# TODO: Could add the mean to the negative values? 
print("[Excitatory] Min and max time constants after:", np.min(exc_syn_time_constants), np.max(exc_syn_time_constants))
print(f"[Excitatory] Mean time constants after: {np.mean(exc_syn_time_constants)} ± {np.std(exc_syn_time_constants)}")

Min and max time constants before: -21.233776011224116 24.174863238143846
[Excitatory] Min and max time constants after: 0.014782422249804306 24.174863238143846
[Excitatory] Mean time constants after: 7.256403369442719 ± 5.2484092838917515


### The inhibitory time constants will be calculated from the excitatory time constants by subtracting a value in a range

In [287]:
# Generate the inhibitory time constants by subtracting a random value in a range from the excitatory time constants

# Generate the random values to subtract from the excitatory time constants
inh_syn_offset = np.random.uniform(inh_subtract_range[0], inh_subtract_range[1], n_lif1)
preview_np_array(inh_syn_offset, "inh_syn_offset", edge_items=10)

# Subtract the random values from the excitatory time constants to get the inhibitory time constants
inh_syn_time_constants = exc_syn_time_constants - inh_syn_offset

# Clip the inhibitory time constants to be positive or equal to the minimum found excitatory time 

inh_syn_time_constants = np.clip(inh_syn_time_constants, a_min=np.min(exc_syn_time_constants), a_max=None)
print("[Inhibitory] Min and max time constants after:", np.min(inh_syn_time_constants), np.max(inh_syn_time_constants))
print(f"[Inhibitory] Mean time constants after: {np.mean(inh_syn_time_constants)} ± {np.std(inh_syn_time_constants)}")

inh_syn_offset Shape: (256,).
Preview: [0.38409005 0.32167634 0.08576844 0.23937426 0.24568982 0.40408243
 0.26415371 0.18323644 0.39491357 0.22807042 ... 0.20199229 0.4155711
 0.27075911 0.28780505 0.12896734 0.05395438 0.09437813 0.4911963
 0.43703813 0.46848766]
[Inhibitory] Min and max time constants after: 0.014782422249804306 23.72087779382895
[Inhibitory] Mean time constants after: 6.989363531691301 ± 5.23801101188967


In [288]:
# Convert the excitatory time constants to fractions (du_exc values) that are used in the LAVA Processes dynamics
exc_syn_time_constants_frac = time_constant_to_fraction(exc_syn_time_constants)
preview_np_array(exc_syn_time_constants_frac, "exc_syn_time_constants_frac", edge_items=5)

print(f"Min. Exc. τ: {np.min(exc_syn_time_constants_frac)}. Max. Exc. τ: {np.max(exc_syn_time_constants_frac)}")
print(f"Mean Exc. τ: {np.mean(exc_syn_time_constants_frac)} ± {np.std(exc_syn_time_constants_frac)}")

# TODO: Maybe there is a better approx. function than normal distribution that avoids having 1.0 time constant (no decay)

exc_syn_time_constants_frac Shape: (256,).
Preview: [0.99999915 0.09309258 0.19678155 0.14793284 0.0937647  ... 0.09586362
 0.35099961 0.1285995  0.17366133 0.14029631]
Min. Exc. τ: 0.04052141293343192. Max. Exc. τ: 1.0
Mean Exc. τ: 0.23631028852270428 ± 0.22529475344864336


In [289]:
# Convert the inhibitory time constants to fractions (du_inh values) that are used in the LAVA Processes dynamics
inh_syn_time_constants_frac = time_constant_to_fraction(inh_syn_time_constants)
preview_np_array(inh_syn_time_constants_frac, "inh_syn_time_constants_frac", edge_items=5)

print(f"Min. Inh. τ: {np.min(inh_syn_time_constants_frac)}. Max. Inh. τ: {np.max(inh_syn_time_constants_frac)}")
print(f"Mean Inh. τ: {np.mean(inh_syn_time_constants_frac)} ± {np.std(inh_syn_time_constants_frac)}")

inh_syn_time_constants_frac Shape: (256,).
Preview: [1.         0.09596393 0.20014579 0.15335116 0.09597382 ... 0.0963616
 0.36282554 0.13725489 0.1878732  0.15014468]
Min. Inh. τ: 0.04128070792441396. Max. Inh. τ: 1.0
Mean Inh. τ: 0.25676760598082016 ± 0.2513100016597133


#### Define the Membrane Potential Time Constants for the LIF Neurons
To add more variability to the network, the membrane potential time constants will be randomly generated from a normal distribution around a mean value.

In [290]:
dv_time_constants = np.random.normal(mean_dv, std_dev_dv, n_lif1)

preview_np_array(dv_time_constants, "dv_time_constants", edge_items=5)

# Guarantee that the time constants are positive
dv_time_constants = np.abs(dv_time_constants)
print("[ms] Min and max Voltage time constants:", np.min(dv_time_constants), np.max(dv_time_constants))

# Transform the time constants to fractions
dv_time_constants_frac = time_constant_to_fraction(dv_time_constants)
print("[Frac] Min and max Voltage time constants:", np.min(dv_time_constants_frac), np.max(dv_time_constants_frac))

dv_time_constants Shape: (256,).
Preview: [12.49211481 20.16052832  9.40646181 12.31258418  7.25476494 ...
 13.67565006 16.23223569 10.6666522   8.26144235 17.00871844]
[ms] Min and max Voltage time constants: 1.6190452380672298 31.152728318306924
[Frac] Min and max Voltage time constants: 0.03159018356562193 0.46078881027393426


In [291]:
from lava.proc.lif.process import ConfigTimeConstantsLIF

configLIF = ConfigTimeConstantsLIF(shape=(n_lif1,),  # There are 256 neurons
            vth=v_th,  # TODO: Verify these initial values
            v=v_init,
            dv=dv_time_constants_frac,    # Inverse of decay time-constant for voltage decay
            du_exc=exc_syn_time_constants_frac,  # Inverse of decay time-constant for excitatory current decay
            du_inh=inh_syn_time_constants_frac,  # Inverse of decay time-constant for inhibitory current decay
            bias_mant=0,
            bias_exp=0,
            name="lif1")

configLIF.du_exc

Variable: du_exc
    shape: (256,)
    init: [0.99999915 0.09309258 0.19678155 0.14793284 0.0937647  ... 0.09586362
 0.35099961 0.1285995  0.17366133 0.14029631]
    shareable: True
    value: [0.99999915 0.09309258 0.19678155 0.14793284 0.0937647  ... 0.09586362
 0.35099961 0.1285995  0.17366133 0.14029631]

### Create the Dense Layers

In [292]:
# Create Dense Process to connect the input layer and LIF1
# create weights of the dense layer
# dense_weights_input = np.eye(N=n1, M=n1)
# Fully Connected Layer from n_spike_gen neurons to n_lif1 neurons
dense_weights_input = np.ones(shape=(n_lif1, n_spike_gen))

# Make the weights (synapses) connecting the odd-parity neurons of the input layer to the network negative (inhibitory)
dense_weights_input[:, 1::2] *= -1

# multiply the weights of the Dense layer by a constant
dense_weights_input *= weights_scale_input
dense_input = Dense(weights=np.array(dense_weights_input), name="DenseInput")

#### Look at the weights of the Dense Layers

In [293]:
# Weights of the Input Dense Layer
dense_input.weights.get()

array([[ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       ...,
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3],
       [ 0.3, -0.3]])

### Map the input channels to the corresponding indexes in the input layer
Since the input channels in the input file may be of any number, we need to **map the input channels to the corresponding indexes in the input layer**. This is done by the `channel_map` dictionaries.

The network expects an UP and DOWN spike train for each channel. Thusly, let's define 2 dictionaries, one for the UP spikes and one for the DOWN spikes. We want the UP and DOWN spike trains to be followed by each other in the input layer for each channel.

In [294]:
# Map the channels of the input file to the respective index in the output list of SpikeEventGen

# Define the mapping of the channels of the UP spike train to the respective index in the output list of SpikeEventGen
up_channel_map = {-1: 0}
# Define the mapping of the channels of the DOWN spike train to the respective index in the output list of SpikeEventGen
down_channel_map = {-1: 1}

# Define constants related to the simulation time

In [295]:
init_offset = 0 # 900 # 33400      #   
virtual_time_step_interval = 1  # TODO: Check if this should be the time-step value. it is not aligned with the sampling rate of the input data

num_steps = 120000    # 200 # Number of steps to run the simulation

# OPTIONAL: Scale down the simulation time
time_downscale = 4  # 

num_steps = num_steps // time_downscale

### Update the UP and DOWN spike trains to include only the spikes that occur within the simulation time

In [296]:
# Iterate the UP spike train to find the spikes that occur within the time interval
up_train_start = -1
up_train_end = up_spike_train.shape[0]
for i, (spike_time, _) in enumerate(up_spike_train):
    if up_train_start == -1 and spike_time >= init_offset:
        up_train_start = i
    
    if spike_time > init_offset + num_steps:
        up_train_end = i
        break

# Slice the spike train to the time interval
up_spike_train_interval = []
if up_train_start != -1:
    # If there are spikes in the interval
    up_spike_train_interval = up_spike_train[up_train_start:up_train_end]
    
preview_np_array(up_spike_train_interval, "Spike Events")

Spike Events Shape: (1792, 2).
Preview: [[ 5.17578125e+01 -1.00000000e+00]
 [ 5.90820312e+01 -1.00000000e+00]
 [ 8.44726562e+01 -1.00000000e+00]
 [ 1.12792969e+02 -1.00000000e+00]
 [ 1.23046875e+02 -1.00000000e+00]
 ...
 [ 2.97636719e+04 -1.00000000e+00]
 [ 2.98291016e+04 -1.00000000e+00]
 [ 2.98808594e+04 -1.00000000e+00]
 [ 2.98886719e+04 -1.00000000e+00]
 [ 2.98901367e+04 -1.00000000e+00]]


In [297]:
# Iterate the UP spike train to find the spikes that occur within the time interval
down_train_start = -1
down_train_end = down_spike_train.shape[0]
for i, (spike_time, _) in enumerate(down_spike_train):
    if down_train_start == -1 and spike_time >= init_offset:
        down_train_start = i
    
    if spike_time > init_offset + num_steps:
        down_train_end = i
        break

# Slice the spike train to the time interval
down_spike_train_interval = []
if down_train_start != -1:
    # If there are spikes in the interval
    down_spike_train_interval = down_spike_train[down_train_start:down_train_end]
    
preview_np_array(down_spike_train_interval, "Spike Events")

Spike Events Shape: (1809, 2).
Preview: [[ 5.46875000e+01 -1.00000000e+00]
 [ 5.66406250e+01 -1.00000000e+00]
 [ 8.15429688e+01 -1.00000000e+00]
 [ 1.06933594e+02 -1.00000000e+00]
 [ 1.20117188e+02 -1.00000000e+00]
 ...
 [ 2.97592773e+04 -1.00000000e+00]
 [ 2.98261719e+04 -1.00000000e+00]
 [ 2.98828125e+04 -1.00000000e+00]
 [ 2.98842773e+04 -1.00000000e+00]
 [ 2.98930664e+04 -1.00000000e+00]]


## Implement the `SpikeEventGenerator` Model

In [298]:
from lava.magma.core.model.py.model import PyLoihiProcessModel  # Processes running on CPU inherit from this class
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.model.py.ports import PyOutPort

@implements(proc=SpikeEventGen, protocol=LoihiProtocol)
@requires(CPU)
class PySpikeEventGenModel(PyLoihiProcessModel):
    """Spike Event Generator Process implementation running on CPU (Python)
    Args:
    """
    s_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)   # IT IS POSSIBLE TO SEND FLOATS AFTER ALL
    exc_spike_events: np.ndarray = LavaPyType(np.ndarray, np.ndarray)
    inh_spike_events: np.ndarray = LavaPyType(np.ndarray, np.ndarray)

    def __init__(self, proc_params) -> None:
        super().__init__(proc_params=proc_params)
        # print("spike events", self.spike_events.__str__())    # TODO: Check why during initialization the variable prints the class, while during run it prints the value
        
        self.curr_exc_idx = 0     # Index of the next excitatory spiking event to send
        self.curr_inh_idx = 0     # Index of the next inhibitory spiking event to send
        self.virtual_time_step_interval = virtual_time_step_interval  # 1000    # Arbitrary time between time steps (in microseconds). This is not a real time interval (1000ms = 1s)
        self.init_offset = init_offset        # 698995               # Arbitrary offset to start the simulation (in microseconds)
        
        # Try to increment the curr_exc_idx and curr_inh_idx to the first spike event that is greater than the init_offset here?

    def run_spk(self) -> None:
        spike_data = np.zeros(self.s_out.shape) # Initialize the spike data to 0
        
        #print("time step:", self.time_step)

        # If the current simulation time is greater than a spike event, send a spike in the corresponding channel
        currTime = self.init_offset + self.time_step*self.virtual_time_step_interval

        spiking_channels = set()   # List of channels that will spike in the current time step

        # Add the excitatory spike events to the spike_date
        while (self.curr_exc_idx < len(self.exc_spike_events)) and currTime >= self.exc_spike_events[self.curr_exc_idx][0]:
            # Get the channel of the current spike event
            curr_channel = self.exc_spike_events[self.curr_exc_idx][1]

            # Check if the channel is valid (belongs to a channel in the up_channel_map therefore it has an output index)
            if curr_channel not in up_channel_map:
                self.curr_exc_idx += 1
                continue    # Skip the current spike event

            # Check if the next spike belongs to a channel that will already spike in this time step
            # If so, we don't add the event and stop looking for more events
            if curr_channel in spiking_channels:
                break

            # Add the channel to the list of spiking channels
            spiking_channels.add(curr_channel)

            # Get the output index of the current channel according to the up_channel_map
            out_idx = up_channel_map[curr_channel]
            if out_idx < self.s_out.shape[0]:   # Check if the channel is valid
                # Update the spike_data with the excitatory spike event (value = 1.0)
                spike_data[out_idx] = 1.0   # Send spike (value corresponds to the punctual current of the spike event)

            # Move to the next spike event
            self.curr_exc_idx += 1

        # Add the inhibitory spike events to the spike_date
        while (self.curr_inh_idx < len(self.inh_spike_events)) and currTime >= self.inh_spike_events[self.curr_inh_idx][0]:
            # Get the channel of the current spike event
            curr_channel = self.inh_spike_events[self.curr_inh_idx][1]

            # Check if the channel is valid (belongs to a channel in the down_channel_map therefore it has an output index)
            if curr_channel not in down_channel_map:
                self.curr_inh_idx += 1
                continue    # Skip the current spike event

            # Check if the next spike belongs to a channel that will already spike in this time step
            # If so, we don't add the event and stop looking for more events
            if curr_channel in spiking_channels:
                break

            # Add the channel to the list of spiking channels
            spiking_channels.add(curr_channel)

            # Get the output index of the current channel according to the down_channel_map
            out_idx = down_channel_map[curr_channel]
            if out_idx < self.s_out.shape[0]:   # Check if the channel is valid
                # It is not possible to send negative values or floats in the spike_data. The weight of the synapse should do the inhibition
                spike_data[out_idx] = 1.0   # Send spike (value corresponds to the punctual current of the spike event)

            # Move to the next spike event
            self.curr_inh_idx += 1


        if len(spiking_channels) > 0:   # Print the spike event if there are any spikes
            VERBOSE = False
            if VERBOSE:
                print(f"""Sending spike event at time: {currTime}({self.time_step}). Last (E/I) spike idx: {self.curr_exc_idx-1}/{self.curr_inh_idx-1}
                        Spike times: {self.exc_spike_events[self.curr_exc_idx-1][0] if self.curr_exc_idx > 0 else "?"}/\
                        {self.inh_spike_events[self.curr_inh_idx-1][0] if self.curr_inh_idx > 0 else "?"}
                        Spike_data: {spike_data}\n"""
                )
            #else:
            #     print(f"Sending spike event at time: {currTime}({self.time_step}).")

        # Send spikes if self.curr_exc_idx > 0 else "?"
        # print("sending spike_data: ", spike_data, " at step: ", self.time_step)
        self.s_out.send(spike_data)

        # Stop the Process if there are no more spike events to send. (It will stop all the connected processes)
        # TODO: Should it be another process that stops the simulation? Such as the last LIF process
        # if self.curr_spike_idx >= 5: # len(self.spike_events):
        #    self.pause()

        # Print a progress message every 1000 time steps
        if self.time_step % 1000 == 0:
            # Clear the console
            print(f"Time step: {self.time_step}")

## Connect the Layers
To define the connectivity between the `SpikeGenerator` and the first `LIF` population, we use another `Dense` Layer.

In [299]:
# Create the Input Process
spike_event_gen = SpikeEventGen(out_shape=(n_spike_gen,),
                                exc_spike_events=up_spike_train_interval,
                                inh_spike_event=down_spike_train_interval,
                                name="SpikeEventsGenerator")

# If I connect the SpikeEventGen to the Dense Layer, the a_out value of the custom input will be rounded to 0 or 1 in the Dense Layer (it will not be a float) 
# However, setting the Dense weights to a float works instead
# Connect the SpikeEventGen to the Dense Layer
spike_event_gen.s_out.connect(dense_input.s_in)

# Connect the Dense_Input to the LIF1 Layer
dense_input.a_out.connect(configLIF.a_in)

### Take a look at the connections in the Input Layer

In [300]:
for proc in [spike_event_gen, dense_input, configLIF]:
    for port in proc.in_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
    for port in proc.out_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")

Proc: SpikeEventsGenerator Port Name: s_out Size: 2
Proc: DenseInput Port Name: s_in  Size: 2
Proc: DenseInput Port Name: a_out Size: 256
Proc: lif1  Port Name: a_in  Size: 256
Proc: lif1  Port Name: s_out Size: 256


### Record Internal Vars over time
To record the evolution of the internal variables over time, we need a `Monitor`. For this example, we want to record the membrane potential of the `LIF` Layer, hence we need 1 `Monitors`.

We can define the `Var` that a `Monitor` should record, as well as the recording duration, using the `probe` function

In [301]:
from lava.proc.monitor.process import Monitor

monitor_lif1_v = Monitor()
monitor_lif1_u = Monitor()

# Connect the monitors to the variables we want to monitor
monitor_lif1_v.probe(configLIF.v, num_steps)
monitor_lif1_u.probe(configLIF.u, num_steps)  # Monitoring the net_current (u_exc + u_inh) of the LIF1 Process

## Execution
Now that we have defined the network, we can execute it. We will use the `run` function to execute the network.

### Run Configuration and Conditions

In [302]:
from lava.magma.core.run_conditions import RunContinuous, RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg

# run_condition = RunContinuous()   # TODO: Change to this one
run_condition = RunSteps(num_steps=num_steps)
run_cfg = Loihi1SimCfg(select_tag="floating_pt")   # TODO: Check why we need this select_tag="floating_pt"

### Execute

In [303]:
configLIF.run(condition=run_condition, run_cfg=run_cfg)

Time step: 1000
Time step: 2000
Time step: 3000
Time step: 4000
Time step: 5000
Time step: 6000
Time step: 7000
Time step: 8000
Time step: 9000
Time step: 10000
Time step: 11000
Time step: 12000
Time step: 13000
Time step: 14000
Time step: 15000
Time step: 16000
Time step: 17000
Time step: 18000
Time step: 19000
Time step: 20000
Time step: 21000
Time step: 22000
Time step: 23000
Time step: 24000
Time step: 25000
Time step: 26000
Time step: 27000
Time step: 28000
Time step: 29000
Time step: 30000


### Retrieve recorded data

In [304]:
data_lif1_v = monitor_lif1_v.get_data()
data_lif1_u = monitor_lif1_u.get_data()

print("Copying...")
data_lif1 = data_lif1_v.copy()
data_lif1["lif1"]["u"] = data_lif1_u["lif1"]["u"]   # Merge the dictionaries to contain both voltage and current


Copying...


In [305]:
configLIF

In [306]:
# Check the shape to verify if it is printing the voltage for every step
print(len(data_lif1['lif1']['v']))     # Indeed, there are 300 values (same as the number of steps we ran the simulation for)

30000


### Plot the recorded data

In [307]:
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

# Boolean defining if we should use the monitor plot
MONITOR_PLOT = False

if MONITOR_PLOT:
    # Create a subplot for each monitored variable
    fig = plt.figure(figsize=(16, 10))
    ax0 = fig.add_subplot(221)
    ax0.set_title('Voltage (V) / time step')
    ax1 = fig.add_subplot(222)
    ax1.set_title('Current (U) / time step')


    # Plot the data
    monitor_lif1_v.plot(ax0, lif1.v)
    monitor_lif1_u.plot(ax1, lif1.u)

## Find the timesteps where the network spiked

In [308]:
from utils.data_analysis import find_spike_times

lif1_voltage_vals = np.array(data_lif1['lif1']['v'])
lif1_current_vals = np.array(data_lif1['lif1']['u'])
# preview_np_array(voltage_arr_1, "Voltage Array")

# Call the find_spike_times util function that detects the spikes in a voltage array
# TODO: Improve the find_spike_times method to view the current of the preview timestep to make sure it is a spike, instead of an inhibition
spike_times_lif1 = find_spike_times(lif1_voltage_vals, lif1_current_vals)

for (spike_time, neuron_idx) in spike_times_lif1:
    print(f"Spike time: {init_offset + spike_time * virtual_time_step_interval} (iter. {spike_time}) at neuron: {neuron_idx}")


Spike time: 268 (iter. 268) at neuron: 1
Spike time: 268 (iter. 268) at neuron: 5
Spike time: 268 (iter. 268) at neuron: 6
Spike time: 268 (iter. 268) at neuron: 7
Spike time: 268 (iter. 268) at neuron: 9
Spike time: 268 (iter. 268) at neuron: 13
Spike time: 268 (iter. 268) at neuron: 18
Spike time: 268 (iter. 268) at neuron: 20
Spike time: 268 (iter. 268) at neuron: 30
Spike time: 268 (iter. 268) at neuron: 35
Spike time: 268 (iter. 268) at neuron: 37
Spike time: 268 (iter. 268) at neuron: 40
Spike time: 268 (iter. 268) at neuron: 45
Spike time: 268 (iter. 268) at neuron: 47
Spike time: 268 (iter. 268) at neuron: 51
Spike time: 268 (iter. 268) at neuron: 56
Spike time: 268 (iter. 268) at neuron: 58
Spike time: 268 (iter. 268) at neuron: 63
Spike time: 268 (iter. 268) at neuron: 64
Spike time: 268 (iter. 268) at neuron: 72
Spike time: 268 (iter. 268) at neuron: 74
Spike time: 268 (iter. 268) at neuron: 75
Spike time: 268 (iter. 268) at neuron: 77
Spike time: 268 (iter. 268) at neuron: 

## View the Voltage and Current dynamics with an interactive plot

Grab the data from the recorded variables

In [309]:
preview_np_array(lif1_voltage_vals, "Voltage Values", edge_items=3)

Voltage Values Shape: (30000, 256).
Preview: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-7.08159320e-06  3.34039981e-03 -3.51739846e-06 ...  9.15927189e-05
   2.67138593e-06  1.32251307e-03]
 [-6.53680434e-06  3.18332691e-03 -3.16264998e-06 ...  8.35440896e-05
   2.36743501e-06  1.24703638e-03]
 [-6.03392624e-06  3.03347061e-03 -2.84368012e-06 ...  7.61975148e-05
   2.09803385e-06  1.17586432e-03]]


## Assemble the values to be plotted

In [310]:
from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
x = [val + init_offset for val in range(num_steps)]

v_y1 = [val[0] for val in lif1_voltage_vals]
v_y2 = [val[27] for val in lif1_voltage_vals]
v_y3 = [val[28] for val in lif1_voltage_vals]
v_y4 = [val[29] for val in lif1_voltage_vals]
v_y5 = [val[4] for val in lif1_voltage_vals]
v_y6 = [val[5] for val in lif1_voltage_vals]
v_y7 = [val[6] for val in lif1_voltage_vals]
v_y8 = [val[7] for val in lif1_voltage_vals]
v_y9 = [val[8] for val in lif1_voltage_vals]
v_y10 = [val[9] for val in lif1_voltage_vals]

# Create the plot
voltage_lif1_y_arrays = [
    (v_y1, "Neuron. 0"), (v_y2, "Neuron. 1"), (v_y3, "Neuron. 2"),
    (v_y4, "Neuron. 3"), (v_y5, "Neuron. 4"), # (v_y6, "Neuron. 5"),
    # (v_y7, "Neuron. 6"), (v_y8, "Neuron. 7"), (v_y9, "Neuron. 8"),
    # (v_y10, "Neuron. 9")
]    # List of tuples containing the y values and the legend label
# Define the box annotation parameters
box_annotation_voltage = {
    "bottom": 0,
    "top": v_th,
    "left": 0,
    "right": num_steps,
    "fill_alpha": 0.03,
    "fill_color": "green"
}

# Create the LIF1 Voltage
voltage_lif1_plot = create_fig(
    title="LIF1 Voltage dynamics", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (V)',
    x=x, 
    y_arrays=voltage_lif1_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    box_annotation_params=box_annotation_voltage,
    y_range=Range1d(-1.05, 1.05)
)


# Create the LIF1 Current
u_y1 = [val[0] for val in lif1_current_vals]
u_y2 = [val[27] for val in lif1_current_vals]
u_y3 = [val[28] for val in lif1_current_vals]
u_y4 = [val[29] for val in lif1_current_vals]
u_y5 = [val[4] for val in lif1_current_vals]
current_lif1_y_arrays = [(u_y1, "Neuron. 0"), (u_y2, "Neuron. 1"), (u_y3, "Neuron. 2"),
                          (u_y4, "Neuron. 3"), (u_y5, "Neuron. 4")]    # List of tuples containing the y values and the legend label
current_lif1_plot = create_fig(
    title="LIF1 Current dynamics", 
    x_axis_label='time (ms)', 
    y_axis_label='Current (U)',
    x=x, 
    y_arrays=current_lif1_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    x_range=voltage_lif1_plot.x_range,    # Link the x-axis range to the voltage plot
)

# bplt.show(voltage_lif1_plot)

## Show the Plots assembled in a grid

In [311]:
import bokeh.plotting as bplt
from bokeh.layouts import gridplot

showPlot = True
if showPlot:
    # Create array of plots to be shown
    plots = [voltage_lif1_plot, current_lif1_plot]

    if len(plots) == 1:
        grid = plots[0]
    else:   # Create a grid layout
        grid = gridplot(plots, ncols=2, sizing_mode="stretch_both")

    # Show the plot
    bplt.show(grid)

## Export the plot to a file

In [312]:
export = False
OUTPUT_FOLDER = f"./results/{DATASET_FILENAME}"
TIME_SUFFIX = f"time{init_offset}-{num_steps}-{virtual_time_step_interval}"
THRESH_SUFFIX = f"thresh{thresh_up}-{thresh_down}"
STRAT_SUFFIX = f"strat{selected_strategy}"
WEIGHT_SUFFIX = f"w{weights_scale_input}"
DV_SUFFIX = f"dv{mean_dv}"
INH_RANGE = f"inh{inh_subtract_range[0]}-{inh_subtract_range[1]}"

if export:
    # Create the output folder if it does not exist
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    file_path = f"{OUTPUT_FOLDER}/{band_file_name}_output_{DV_SUFFIX}_{WEIGHT_SUFFIX}_{THRESH_SUFFIX}_{INH_RANGE}_{STRAT_SUFFIX}_{TIME_SUFFIX}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="HFO Detection - Voltage and Current dynamics")

    # Save the plot
    bplt.save(grid)

## Export the Voltage and Current dynamics to a `.npy` file
In order to classify the feature neurons (Noisy, Silent, Ripple, or Fast Ripple Detector), we need to export the voltage and current dynamics to a `.npy` file to be analyzed by a Classification Algorithm

In [313]:
EXPORT_DYNAMICS = True
if EXPORT_DYNAMICS:
    # Define the file paths to save the Voltage and Current dynamics
    v_dynamics_file_path = f"{OUTPUT_FOLDER}/{band_file_name}_v_dynamics_{DV_SUFFIX}_{WEIGHT_SUFFIX}_{THRESH_SUFFIX}_{INH_RANGE}_{STRAT_SUFFIX}_{TIME_SUFFIX}.npy"
    u_dynamic_file_path = f"{OUTPUT_FOLDER}/{band_file_name}_u_dynamics_{DV_SUFFIX}_{WEIGHT_SUFFIX}_{THRESH_SUFFIX}_{INH_RANGE}_{STRAT_SUFFIX}_{TIME_SUFFIX}.npy"
    
    # Export the Voltage dynamics to a numpy file
    np.save(v_dynamics_file_path, lif1_voltage_vals)
    
    # Export the Current dynamics to a numpy file
    np.save(u_dynamic_file_path, lif1_current_vals)

## Export the Ground Truth data to a `.npy` file along with necessary Simulation Parameters

### Load the Ground Truth data from the `.npy` file

In [314]:
# Load the ground_truth data
ground_truth_file_name = f"{INPUT_PATH}/{band_file_name}_ground_truth.npy"

ground_truth = np.load(ground_truth_file_name)

preview_np_array(ground_truth, "ground_truth", edge_items=3)
print(f"Number of relevant events: {np.count_nonzero(ground_truth)}")

ground_truth Shape: (222,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]
Number of relevant events: 222


In [315]:
from utils.snn import SNNSimConfig

EXPORT_CONFIG = True
if EXPORT_CONFIG:
    # Define the simulation configuration
    snn_config = SNNSimConfig(ground_truth, init_offset, virtual_time_step_interval, num_steps)

    snn_config_file_name = f"{OUTPUT_FOLDER}/{band_file_name}_snn_config_{TIME_SUFFIX}.npy"
    # Save the SNN Config Class to a npy file
    np.save(snn_config_file_name, snn_config)

## Stop the Runtime

In [316]:
lif1.stop()